# Environment

## install libraries, import dependencies, set global variables, and integrate utilities, i.e. helper classes and methods

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!pip install "ibm-watson-machine-learning>=1.0.321" | tail -n 1

In [ ]:
import ibm_watson_machine_learning.foundation_models as foundation_models
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model

import os
import json
from datetime import datetime

# Helper Methods
# @TODO Add error handling and log implementation
def load_environment_variables(env_file_path):
    """
    Load environment variables from a file.

    Args:
    - env_file_path (str): Path to the file containing environment variables.

    Returns:
    - dict: Dictionary containing the environment variables.
    """
    env_vars = {}

    with open(env_file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=', 1)
            env_vars[key] = value

    return env_vars

def get_prompt_from_file(file_path):
    with open(file_path, 'r') as file:
        file_contents = file.read()
    return file_contents

def store_configuration_checkpoint(iQ_config):
    timestamp = datetime.now().strftime('%d%b%Y_%H%M')
    filename = f"{timestamp}_iQ_config.json"
    full_path = os.path.join(PATH_TO_CHECKPOINT_STORE, filename)

    with open(full_path, 'w') as f:
        json.dump(iQ_config, f, indent=4)

# Environmental and Global Variables
# @TODO troubleshoot .env file integration issue
# @TODO convert this to a manager class --> iQ_MANAGER, instantiate before User Form
ENV_PATH = '/content/gdrive/MyDrive/Colab Notebooks/iQ/.env'
environment_variables = load_environment_variables(ENV_PATH)
WATSON_API_KEY = environment_variables.get("WATSON_API_KEY")
WATSON_URL = environment_variables.get("WATSON_URL")
PROJECT_ID = environment_variables.get("PROJECT_ID")
MODEL_ID = "meta-llama/llama-2-70b-chat"
CREDS = {
    "url": WATSON_URL,
    "apikey": WATSON_API_KEY
}
PARAMS = {
  "decoding_method": "greedy",
  "max_new_tokens": 20,
  "min_new_tokens": 0,
  "stop_sequences": [],
  "repetition_penalty": 1
 }
model = Model(
    model_id=MODEL_ID,
    params=PARAMS,
    credentials=CREDS,
    project_id=PROJECT_id)

PATH_TO_PROMPT_LIBRARY = "/content/gdrive/MyDrive/Colab Notebooks/iQ/prompt_library"
PATH_TO_CHECKPOINT_STORE = "/content/gdrive/MyDrive/Colab Notebooks/iQ/checkpoint_store"

# Users can add COT prompts to their own personally curated Prompt Library.  The Prompt library is a Technical Manual for the AI Agent to gain insight into how to utilize the tools in the iQ_config "tools" section.
PROMPT_LIBRARY = {
    "INIT": "initPrompt.txt",
    "USER_CONFIG": "firstPrompt.txt",
    "BEGIN_LESSON": "firstPrompt.txt",
    "CONCLUDE_LESSON": "firstPrompt.txt"
}



# Phase 1 Implementation:

In [ ]:
#@title User Configuration

# Provide by User
user_name = 'Anthem' #@param {type:"string"}
expertise_level =  'beginner' #@param ["beginner", "novice", "expert"] {allow-input: true}
learning_depth = 'deep dive' #@param ["overview", "detailed", "deep dive"] {allow-input: true}
time_in_minutes = '30' #@param [5, 10, 15, 30, 45, 60] {allow-input: true}
execute_tool = 'INIT' #@param ["INIT", "USER_CONFIG", "BEGIN_LESSON", "CONCLUDE_LESSON"] {allow-input: true}
subject_matter = 'Quantum Computing' #@param {type:"string"}

# Generated by iQ
burndown_list = {} # key=task, value=status
praxis = {} # key=task, value=instruction, reinformcement, and synthesis generated by iQ

# Prompt Library
current_prompt = get_prompt_from_file(f"{PATH_TO_PROMPT_LIBRARY}/{PROMPT_LIBRARY[execute_tool]}")

# Configuration
iQ_config = {
    "config": {
        "version": "1.0",
        "purpose": "Human-AI Collaboration for Skill Transfer and Skill Adoption",
        "description": "This configuration is designed to optimize the collaboration between a user and an AI agent for the purpose of learning and mastery of specific subject matter."
    },
    "user": {
        "name": user_name,
        "expertise_level": expertise_level,
        "learning_depth": learning_depth,
        "time_available": time_in_minutes,
        "execute_tool": execute_tool,
        "subject_matter": subject_matter,
        "tasks": burndown_list,
        "praxis": praxis
    },
    "collaboration": {
        "session": {
            "id": "{auto-incremented based on sessions}",
            "title": "{determined by determineTitle function}",
            "learning_outcome": "{formulated by formulateLearningOutcome function}",
            "tasks": "{devised by deviseTasks function}",
            "feedback": {
                "session_id": "{session_id}",
                "rating": "{user_rating}",
                "comments": "{user_comments}",
                "suggestions": "{user_suggestions}"
            }
        },
        "functions": {
            "incrementSessionId": "Calculate the number of sessions already present and increment by one for the new session.",
            "determineTitle": "Based on your instructional needs, interest, and expertise level, I suggest the title: '{sessionTitle}'. Does that sound appropriate?",
            "formulateLearningOutcome": "Considering your inputs, the learning outcome for this session will be: '{learningOutcome}'",
            "deviseTasks": "To achieve the learning outcome, I recommend the following tasks: {taskList}"
        },
        "interviewQuestions": {
            "subjectQuery": "Welcome {user_name}, what do you want to learn today?",
            "timeQuery": "How much time do you have for this session?",
            "expertiseLevelQuery": "Please specify your current level of understanding: beginner, novice, or expert.",
            "depthQuery": "How deep would you like to delve into this topic? (surface-level overview, detailed study, or comprehensive deep dive)"
        },
        "taskStructure": {
            "instruct": "I will present the material related to '{sessionTitle}' in a structured manner, covering key concepts and providing real-world examples.",
            "test": "After the instruction, I'll assess your understanding with a brief test. Aim for at least 95% accuracy to ensure thorough comprehension.",
            "synthesize": "Finally, I'll prompt you to synthesize new information based on what you've learned. This will help in solidifying your understanding and encourage creative application of the knowledge."
        }
    },
    "sessions": [
        {
            "session_id": "1",
            "timestamp": "{timestamp_of_session}",
            "feedback": "{feedback_provided_by_user}"
        }
    ],
    "tools": {
        "INIT": {
            "description": "Initialize the AI agent and provide an overview of its capabilities.",
            "instructions": "{current_prompt}"
        },
        "USER_CONFIG": {
            "description": "Configure user information and preferences for the session.",
            "actions": [
                "{execute: collaboration.interviewQuestions.subjectQuery}",
                "{execute: collaboration.interviewQuestions.timeQuery}",
                "{execute: collaboration.interviewQuestions.expertiseLevelQuery}",
                "{execute: collaboration.interviewQuestions.depthQuery}"
            ]
        },
        "BEGIN_LESSON": {
            "description": "Starts the lesson execution process.",
            "actions": [
                "{execute: collaboration.functions.determineTitle}",
                "{execute: collaboration.functions.formulateLearningOutcome}",
                "{execute: collaboration.taskStructure.instruct}",
                "{execute: collaboration.taskStructure.test}",
                "{execute: collaboration.taskStructure.synthesize}"
            ]
        },
        "CONCLUDE_LESSON": {
            "description": "Concludes the lesson and gathers feedback.",
            "actions": [
                "Thank you for participating in this session. Let's wrap things up.",
                "Recording current timestamp for the session.",
                "Please provide your feedback on the session.",
                "{execute: collaboration.session.feedback}"
            ]
        }
    }
}



## Execution testing

In [ ]:
# Configure User
execute_tool = "USER_CONFIG"
iQ_config["user"] = model.generate_text(iQ_config)

# Update checkpoints
store_configuration_checkpoint(iQ_config)

In [ ]:
# Initialize Agent
execute_tool = "INIT"
burndown_list = model.generate_text(iQ_config)

# Update checkpoints
store_configuration_checkpoint(iQ_config)

In [ ]:
# Begin Lesson
execute_tool = "BEGIN_LESSON"
praxis = model.generate_text(iQ_config)

# Update checkpoints
store_configuration_checkpoint(iQ_config)

In [ ]:
# Conclude Lesson
execute_tool = "CONCLUDE_LESSON"
iQ_config["collaboration"].append(model.generate_text(iQ_config))

# Update checkpoints
store_configuration_checkpoint(iQ_config)

# Phase 2 Plan:

Transfer to Build Infrastructure

- FastAPI Uvicorn server, no front-end
  - classes:
    - UserProfile:
    - ManagementCOT: COT prompt curation for management of agents
      - init: init prompt for manager or specialized agent
      - query: COT prompt curation for queries to manager or specialized agent, needs
  - Endpoints:
    - api/start @PUT --> receives str: "start", begins get_user_profile method
    - api/start @POST --> receives str: user_response
    - api/start @GET --> loads user  profile checkpoints
  - methods:
    - audio_output(response_text)
    - audio_input():
      - open connection for steaming input from mic at user command
      - store in user_input
      - close connection
      - return user_input
    - get_user_profile:
      - instantiate UserProfile class
      - user_profile.interview = dict: (str: question, str: response)
      - for question in userprofile.interview
        - current_question = question
        - current_question_audio = text2speech(current_question)
        - audio_output(current_question_audio)
        - user_input = audio_input()
        - user_profile.interview.currentquestion = speech2text(user_input)
    - manage_agent(str: agent_selection, str: prompt_type)
      - switch case: use agent_selection variable as condition to determine which agent to initialize, i.e. manager or specialized --> set isManager to true or false
      - switch case: use prompt_type variable as condition to determine which type of prompting request to ssend, i.e. init or query --> set isInit to true or false    
      - if isInit:
          response = send_request(class_selection.init_prompt)
        else:
          request = encapsulate_query(class_selection.query_prompt)
          response = send_request(request)
    - encapsulate_query(capsule)
      - user_request = audio_input
      - prompt = capsule + user_request
      - send_request(prompt)
    - store_profile_checkpoints
    - load_profile_checkpoints
- IBM Cloud integration
  - Docker Containerization: Dockerfile, docker-compose.yml
  - Create cluster on Kubernetes Container Registry <-- needs Research
  - Deploy container on Ubuntu Virtual Server <-- needs Research



## Workflow

- Create comprehensive, step-by-step detailed action-oriented subject matter lessons in the forms of tasks with high granularity

## Get Context

In [ ]:
def capture_audio():
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True)
    frames = []
    while True:
        data = stream.read(1024)
        frames.append(data)
        if len(frames) >= 100:
            break
    stream.stop_stream()
    stream.close()
    p.terminate()
    return frames

def transcribe_audio(frames):
    client = deepgram.Client()
    transcript = client.transcribe(audio_data=b''.join(frames))
    return transcript.get_text()

def interact_with_watson(prompt_text):
    # Create the foundation model object
    foundation_model = foundation_models.Model(model_id="google/flan-t5-xxl")

    # Generate a response using Watson
    response = foundation_model.generate(prompt=prompt_text)
    return response["generated_text"]

def main_interaction():
    audio_frames = capture_audio()
    user_prompt = f"Encapsulated prompt text {transcribe_audio(audio_frames)}"
    watson_response = interact_with_watson(user_prompt)
    # @TODO: Convert watson_response to audio and play it
    return watson_response


### Text to Speech --> interview questions

### audio output

### audio input

### Speech to Text --> user responses

### refine

- create task from response
- prioritize
- store in user_profile.burndown_list

### iterate

## Generate Output

### Select task based on user_profile.user_availability and user_profile.burndown_list --> selected_task

### Encapsulate user_profile.burndownlist.selected_task within Init Prompt for AI agent --> user_prompt

### Tokenize and send request to Watson

### Text to Speech <-- response

### Audio output <-- response

### Store Checkpoints <-- response

## Restructure Build Environment:

**Considerations for installing poetry dependency management**

> Running poetry commands without the --no-ansi option fails and does not install the dependencies. So `!poetry --no-ansi install` and `!poetry --no-ansi add package` should be used instead which also correctly updates the pyproject.toml file.

In [ ]:
!pip install deepgram-sdk

In [ ]:
!apt install python3-pyaudio

In [ ]:
!pip install black

In [ ]:
!pip install "black[jupyter]"

In [ ]:
jupyter nbextension enable black/extension

In [ ]:
%cd /content/gdrive/MyDrive/
!rm -rf test-poetry
!mkdir test-poetry
%cd test-poetry

In [ ]:
import deepgram
import pyaudio
import doctest